### Evaluating the SVD++ implementation

In [1]:
import pandas as pd

from surprise import SVDpp
from surprise import Dataset
from surprise import Reader
from surprise import dump
from surprise.model_selection import GridSearchCV
import gc

In [2]:
gc.collect()

0

In [3]:
DATASET_PATH = '../../../datasets'
!ls $DATASET_PATH

books		    books_ratings.csv  refined_metadata.pkl
books_lexile.tar    lexile.json        refined_ratings.csv
books_meta.json.gz  lexile.pkl	       svdpp_dump
books_meta.pkl	    merged_df.pkl      utility_df.pkl


In [4]:
reader = Reader(line_format='user item rating timestamp', sep=',')
data = Dataset.load_from_file(DATASET_PATH+'/refined_ratings.csv', reader=reader)

In [5]:
param_grid = {'n_epochs': [5, 10, 20], 'lr_all': [0.007, 0.009],
              'reg_all': [0.02, 0.4]}
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)
# best RMSE score
print(gs.best_score['rmse'])
# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

0.805552426435
{'lr_all': 0.009, 'reg_all': 0.4, 'n_epochs': 20}


In [7]:
trainset = data.build_full_trainset()
algo = gs.best_estimator['rmse']
algo.fit(trainset)

In [8]:
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

In [14]:
predictions[0]

Prediction(uid='A2DJV5P2NTWEK0', iid='0001981692', r_ui=4.3047654925598042, est=4.1914304167129579, details={u'was_impossible': False})

In [8]:
dump.dump(DATASET_PATH+"/svdpp_dump", algo, predictions)

NameError: name 'predictions' is not defined

In [20]:
with open(DATASET_PATH+"/predictions.csv", "w") as f:
    for prediction in predictions:
        line = str(prediction.uid) + ',' + str(prediction.iid) +',' +\
        str(prediction.est)+"\n"
        f.write(line)

In [17]:
prediction = predictions[0]


In [18]:
line

'A2DJV5P2NTWEK0,0001981692,4.19143041671\n'